# Simple Benchmark Analysis
This notebook demonstrates how you can analyze and plot benchmark results from a single benchmark run.
Several projects exist in the `examples` folder, but this notebook assumes we are working on the
JVM part of the `kotlin-multiplatform` project. But the same approach can be used for the other projects.

First, you need to run the benchmark. This can be done by running this command from the root of the project:

```shell
./gradlew :examples:kotlin-multiplatform:jvmBenchmark
```

Once it is completed, run this notebook, and it will automatically find the latest result.

In [1]:
%use serialization, dataframe, kandy

In [2]:
import java.nio.file.Files
import java.nio.file.attribute.BasicFileAttributes
import kotlin.io.path.exists
import kotlin.io.path.forEachDirectoryEntry
import kotlin.io.path.isDirectory
import kotlin.io.path.listDirectoryEntries
import kotlin.io.path.readText

// Find latest result file, based on the their timestamp.
val runsDir = notebook.workingDir.resolve("kotlin-multiplatform/build/reports/benchmarks/main")
val lastRunDir = runsDir.listDirectoryEntries()
    .filter { it.isDirectory() }
    .sortedByDescending { dir -> Files.readAttributes(dir, BasicFileAttributes::class.java).creationTime() }
    .first()
val outputFile = lastRunDir.resolve("jvm.json")
val benchmarkData = outputFile.readText().deserializeJson()

In [3]:
// Helper class for tracking the information we need to use.
data class Benchmark(val name: String, val params: String, val score: Double, val error: Double, val unit: String)

// Split benchmark results into groups. Generally, each group consist of all tests from one test file,
// except when it is an parameterized test. In this case, each test (with all its variants) are put
// in its own group.
val benchmarkGroups = benchmarkData
    .groupBy {
        if (it.params != null) {
            it.benchmark
        } else {
            it.benchmark.substringBeforeLast(".")
        }
    }
    .mapValues { group ->
        val benchmarks = group.value.map { benchmark ->
            // Parameters are specific to each test. `deserializeJson()` will generate the appropriate data classes,
            // but for generic handling of parameters we would need to fallback to reading the JSON. In this case
            // we just handle them through the typed API.
            val paramInfo = benchmark.params?.let { params -> "data=${params.data},text=\"${params.text}\",value=${params.value}"} ?: ""
            val name = benchmark.benchmark
            Benchmark(
                name,
                paramInfo,
                benchmark.primaryMetric.score,
                benchmark.primaryMetric.scoreError,
                benchmark.primaryMetric.scoreUnit
            )
        }
        benchmarks.toDataFrame()
    }

// Un-commont this to see the benchmark data as DataFrames
// benchmarkGroups.forEach {
//     DISPLAY(it.value)
// }

In [4]:
// Prepare the data frames for plotting by:
// - Add calculated columns for errorMin / errorMax
// - Tests with parameters use the parameter values as the label
// - Tests without paramaters use the test name as the label
val plotData = benchmarkGroups.mapValues {
    it.value
        .add("errorMin") { it.getValue<Double>("score") - it.getValue<Double>("error") }
        .add("errorMax") { it.getValue<Double>("score") + it.getValue<Double>("error") }
        .insert("label") {
            // Re-format the benchmark labels to make them look "nicer"
            if (!it.getValue<String>("params").isBlank()) {
                it.getValue<String>("params").replace(",", "\n")
            } else {
                it.getValue<String>("name").substringAfterLast(".").removeSuffix("Benchmark")
            }
        }.at(0)
        .remove("name", "params")
}

In [5]:
import org.jetbrains.letsPlot.Geom
import org.jetbrains.letsPlot.core.spec.plotson.coord
import org.jetbrains.letsPlot.themes.margin

// Plot each group as a bar plot with the error displayed as error bars.
// This approach assumes that each group has tests roughly within the same "scale".
// If this is not the case, some plots might look very squished. If this happens,
// you can play around with using a LOG10 scale or modifying the limits to focus
// on the changes.
plotData.forEach { (fileName, dataframe) ->
    val plot = dataframe.plot {
        bars {
            x("label") {
                axis.name = ""
            }
            y("score")
        }
        errorBars {
            x("label")
            y("score")
            yMin("errorMin")
            yMax("errorMax")
        }
        coordinatesTransformation = CoordinatesTransformation.cartesianFlipped()
        // y.axis.limits = dataframe.min("errorMin")..dataframe.max("errorMax")
        layout {
            this.yAxisLabel = dataframe.first().getValue<String>("unit")
            style {
                global {
                    title {
                        margin(10.0, 0.0)
                    }
                    text {
                        fontFamily = FontFamily.MONO
                    }
                }
            }
            // Adjust the height of the Kandy plot based on the number of tests.
            size = 800 to ((50 * dataframe.size().nrow) + 100)
        }
    }
    DISPLAY(HTML("<h4 >$fileName</h4>"))
    DISPLAY(plot)
}

test.InheritedBenchmark

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; padding: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="uxz1MH"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 const forceImmediateRender = false;
 const responsive = false;
 
 let sizing = {
 width_mode: "FIXED",
 height_mode: "FIXED",
 width: 800.0, 
 height: 200.0 
 };
 
 const preferredWidth = document.body.dataset.letsPlotPreferredWidth;
 if (preferredWidth !== undefined) {
 sizing = {
 width_mode: 'FIXED',
 height_mode: 'SCALED',
 width: parseFloat(preferredWidth)
 };
 }
 
 const containerDiv = document.getElementById("uxz1MH");
 let fig = null;
 
 function renderPlot() {
 if (fig === null) {
 const plotSpec = {
"mapping":{
},
"guides":{
"y":{
"title":"ops/s"
}
},
"coord":{
"name":"flip",
"flip":true
},
"data":{
"score":[1104972.6706894366,1.4634989745609665E8],
"errorMax":[1154281.987920151,1.489669832012015E8],
"label":["base","inherited"],
"errorMin":[1055663.3534587221,1.437328117109918E8]
},
"ggsize":{
"width":800.0,
"height":200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"score",
"x":"label"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"score",
"x":"label",
"ymin":"errorMin",
"ymax":"errorMax"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data":{
}
}],
"theme":{
"text":{
"family":"mono",
"blank":false
},
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"label"
},{
"type":"float",
"column":"score"
},{
"type":"float",
"column":"errorMin"
},{
"type":"float",
"column":"errorMax"
}]
},
"spec_id":"2"
};
 fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, containerDiv, sizing);
 } else {
 fig.updateView({});
 }
 }
 
 const renderImmediately = 
 forceImmediateRender || (
 sizing.width_mode === 'FIXED' && 
 (sizing.height_mode === 'FIXED' || sizing.height_mode === 'SCALED')
 );
 
 if (renderImmediately) {
 renderPlot();
 }
 
 if (!renderImmediately || responsive) {
 // Set up observer for initial sizing or continuous monitoring
 var observer = new ResizeObserver(function(entries) {
 for (let entry of entries) {
 if (entry.contentBoxSize && 
 entry.contentBoxSize[0].inlineSize > 0) {
 if (!responsive && observer) {
 observer.disconnect();
 observer = null;
 }
 renderPlot();
 if (!responsive) {
 break;
 }
 }
 }
 });
 
 observer.observe(containerDiv);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2e+7 
 
 
 
 
 
 
 4e+7 
 
 
 
 
 
 
 6e+7 
 
 
 
 
 
 
 8e+7 
 
 
 
 
 
 
 1e+8 
 
 
 
 
 
 
 1.2e+8 
 
 
 
 
 
 
 1.4e+8 
 
 
 
 
 
 
 
 
 
 
 base 
 
 
 
 
 
 
 
 
 inherited 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ops/s

test.ParamBenchmark.mathBenchmark

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; padding: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="hsgStz"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 const forceImmediateRender = false;
 const responsive = false;
 
 let sizing = {
 width_mode: "FIXED",
 height_mode: "FIXED",
 width: 800.0, 
 height: 300.0 
 };
 
 const preferredWidth = document.body.dataset.letsPlotPreferredWidth;
 if (preferredWidth !== undefined) {
 sizing = {
 width_mode: 'FIXED',
 height_mode: 'SCALED',
 width: parseFloat(preferredWidth)
 };
 }
 
 const containerDiv = document.getElementById("hsgStz");
 let fig = null;
 
 function renderPlot() {
 if (fig === null) {
 const plotSpec = {
"mapping":{
},
"guides":{
"y":{
"title":"ops/ms"
}
},
"coord":{
"name":"flip",
"flip":true
},
"data":{
"score":[213019.57841385395,214917.7241256119,102424.75929129767,101173.75249296638],
"errorMax":[221389.10361238578,221219.12070789465,103736.05191104818,102438.55849807907],
"label":["data=1\ntext=\"a \"string\" with quotes\"\nvalue=1","data=1\ntext=\"a \"string\" with quotes\"\nvalue=2","data=2\ntext=\"a \"string\" with quotes\"\nvalue=1","data=2\ntext=\"a \"string\" with quotes\"\nvalue=2"],
"errorMin":[204650.0532153221,208616.32754332913,101113.46667154715,99908.94648785368]
},
"ggsize":{
"width":800.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"score",
"x":"label"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"score",
"x":"label",
"ymin":"errorMin",
"ymax":"errorMax"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data":{
}
}],
"theme":{
"text":{
"family":"mono",
"blank":false
},
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"label"
},{
"type":"float",
"column":"score"
},{
"type":"float",
"column":"errorMin"
},{
"type":"float",
"column":"errorMax"
}]
},
"spec_id":"5"
};
 fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, containerDiv, sizing);
 } else {
 fig.updateView({});
 }
 }
 
 const renderImmediately = 
 forceImmediateRender || (
 sizing.width_mode === 'FIXED' && 
 (sizing.height_mode === 'FIXED' || sizing.height_mode === 'SCALED')
 );
 
 if (renderImmediately) {
 renderPlot();
 }
 
 if (!renderImmediately || responsive) {
 // Set up observer for initial sizing or continuous monitoring
 var observer = new ResizeObserver(function(entries) {
 for (let entry of entries) {
 if (entry.contentBoxSize && 
 entry.contentBoxSize[0].inlineSize > 0) {
 if (!responsive && observer) {
 observer.disconnect();
 observer = null;
 }
 renderPlot();
 if (!responsive) {
 break;
 }
 }
 }
 });
 
 observer.observe(containerDiv);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 50,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 150,000 
 
 
 
 
 
 
 200,000 
 
 
 
 
 
 
 
 
 
 
 data=1 
 
 
 text="a "string" with quotes" 
 
 
 value=1 
 
 
 
 
 
 
 
 
 data=1 
 
 
 text="a "string" with quotes" 
 
 
 value=2 
 
 
 
 
 
 
 
 
 data=2 
 
 
 text="a "string" with quotes" 
 
 
 value=1 
 
 
 
 
 
 
 
 
 data=2 
 
 
 text="a "string" with quotes" 
 
 
 value=2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ops/ms

test.ParamBenchmark.otherBenchmark

...

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; padding: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Z7V1TM"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 const forceImmediateRender = false;
 const responsive = false;
 
 let sizing = {
 width_mode: "FIXED",
 height_mode: "FIXED",
 width: 800.0, 
 height: 150.0 
 };
 
 const preferredWidth = document.body.dataset.letsPlotPreferredWidth;
 if (preferredWidth !== undefined) {
 sizing = {
 width_mode: 'FIXED',
 height_mode: 'SCALED',
 width: parseFloat(preferredWidth)
 };
 }
 
 const containerDiv = document.getElementById("Z7V1TM");
 let fig = null;
 
 function renderPlot() {
 if (fig === null) {
 const plotSpec = {
"mapping":{
},
"guides":{
"y":{
"title":"ops/ms"
}
},
"coord":{
"name":"flip",
"flip":true
},
"data":{
"score":[145509.90114707965],
"errorMax":[147776.90932420595],
"label":["math"],
"errorMin":[143242.89296995336]
},
"ggsize":{
"width":800.0,
"height":150.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"score",
"x":"label"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"score",
"x":"label",
"ymin":"errorMin",
"ymax":"errorMax"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data":{
}
}],
"theme":{
"text":{
"family":"mono",
"blank":false
},
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"label"
},{
"type":"float",
"column":"score"
},{
"type":"float",
"column":"errorMin"
},{
"type":"float",
"column":"errorMax"
}]
},
"spec_id":"14"
};
 fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, containerDiv, sizing);
 } else {
 fig.updateView({});
 }
 }
 
 const renderImmediately = 
 forceImmediateRender || (
 sizing.width_mode === 'FIXED' && 
 (sizing.height_mode === 'FIXED' || sizing.height_mode === 'SCALED')
 );
 
 if (renderImmediately) {
 renderPlot();
 }
 
 if (!renderImmediately || responsive) {
 // Set up observer for initial sizing or continuous monitoring
 var observer = new ResizeObserver(function(entries) {
 for (let entry of entries) {
 if (entry.contentBoxSize && 
 entry.contentBoxSize[0].inlineSize > 0) {
 if (!responsive && observer) {
 observer.disconnect();
 observer = null;
 }
 renderPlot();
 if (!responsive) {
 break;
 }
 }
 }
 });
 
 observer.observe(containerDiv);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 60,000 
 
 
 
 
 
 
 80,000 
 
 
 
 
 
 
 100,000 
 
 
 
 
 
 
 120,000 
 
 
 
 
 
 
 140,000 
 
 
 
 
 
 
 
 
 
 
 math 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ops/ms

test.CommonBenchmark

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; padding: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="H3uDBw"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 const forceImmediateRender = false;
 const responsive = false;
 
 let sizing = {
 width_mode: "FIXED",
 height_mode: "FIXED",
 width: 800.0, 
 height: 250.0 
 };
 
 const preferredWidth = document.body.dataset.letsPlotPreferredWidth;
 if (preferredWidth !== undefined) {
 sizing = {
 width_mode: 'FIXED',
 height_mode: 'SCALED',
 width: parseFloat(preferredWidth)
 };
 }
 
 const containerDiv = document.getElementById("H3uDBw");
 let fig = null;
 
 function renderPlot() {
 if (fig === null) {
 const plotSpec = {
"mapping":{
},
"guides":{
"y":{
"title":"ms/op"
}
},
"coord":{
"name":"flip",
"flip":true
},
"data":{
"score":[8.972137714852465E-4,2.369188528309145E-5,6.7916806977233165E-6],
"errorMax":[9.206484453035511E-4,2.4221992778145036E-5,6.8730920645926235E-6],
"label":["long","longBlackhole","math"],
"errorMin":[8.737790976669418E-4,2.3161777788037864E-5,6.710269330854009E-6]
},
"ggsize":{
"width":800.0,
"height":250.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"score",
"x":"label"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"score",
"x":"label",
"ymin":"errorMin",
"ymax":"errorMax"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data":{
}
}],
"theme":{
"text":{
"family":"mono",
"blank":false
},
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"label"
},{
"type":"float",
"column":"score"
},{
"type":"float",
"column":"errorMin"
},{
"type":"float",
"column":"errorMax"
}]
},
"spec_id":"17"
};
 fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, containerDiv, sizing);
 } else {
 fig.updateView({});
 }
 }
 
 const renderImmediately = 
 forceImmediateRender || (
 sizing.width_mode === 'FIXED' && 
 (sizing.height_mode === 'FIXED' || sizing.height_mode === 'SCALED')
 );
 
 if (renderImmediately) {
 renderPlot();
 }
 
 if (!renderImmediately || responsive) {
 // Set up observer for initial sizing or continuous monitoring
 var observer = new ResizeObserver(function(entries) {
 for (let entry of entries) {
 if (entry.contentBoxSize && 
 entry.contentBoxSize[0].inlineSize > 0) {
 if (!responsive && observer) {
 observer.disconnect();
 observer = null;
 }
 renderPlot();
 if (!responsive) {
 break;
 }
 }
 }
 });
 
 observer.observe(containerDiv);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 0.0001 
 
 
 
 
 
 
 0.0002 
 
 
 
 
 
 
 0.0003 
 
 
 
 
 
 
 0.0004 
 
 
 
 
 
 
 0.0005 
 
 
 
 
 
 
 0.0006 
 
 
 
 
 
 
 0.0007 
 
 
 
 
 
 
 0.0008 
 
 
 
 
 
 
 0.0009 
 
 
 
 
 
 
 
 
 
 
 long 
 
 
 
 
 
 
 
 
 longBlackhole 
 
 
 
 
 
 
 
 
 math 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ms/op

test.JvmTestBenchmark

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; padding: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="KErhGY"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 const forceImmediateRender = false;
 const responsive = false;
 
 let sizing = {
 width_mode: "FIXED",
 height_mode: "FIXED",
 width: 800.0, 
 height: 200.0 
 };
 
 const preferredWidth = document.body.dataset.letsPlotPreferredWidth;
 if (preferredWidth !== undefined) {
 sizing = {
 width_mode: 'FIXED',
 height_mode: 'SCALED',
 width: parseFloat(preferredWidth)
 };
 }
 
 const containerDiv = document.getElementById("KErhGY");
 let fig = null;
 
 function renderPlot() {
 if (fig === null) {
 const plotSpec = {
"mapping":{
},
"guides":{
"y":{
"title":"ns/op"
}
},
"coord":{
"name":"flip",
"flip":true
},
"data":{
"score":[3.5445401759264423,0.5428283650466172],
"errorMax":[3.572058960798929,0.5468378157356791],
"label":["cos","sqrt"],
"errorMin":[3.5170213910539556,0.5388189143575554]
},
"ggsize":{
"width":800.0,
"height":200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"score",
"x":"label"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
"y":"score",
"x":"label",
"ymin":"errorMin",
"ymax":"errorMax"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"errorbar",
"data":{
}
}],
"theme":{
"text":{
"family":"mono",
"blank":false
},
"title":{
"margin":[10.0,0.0,10.0,0.0],
"blank":false
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
},
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"label"
},{
"type":"float",
"column":"score"
},{
"type":"float",
"column":"errorMin"
},{
"type":"float",
"column":"errorMax"
}]
},
"spec_id":"20"
};
 fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, containerDiv, sizing);
 } else {
 fig.updateView({});
 }
 }
 
 const renderImmediately = 
 forceImmediateRender || (
 sizing.width_mode === 'FIXED' && 
 (sizing.height_mode === 'FIXED' || sizing.height_mode === 'SCALED')
 );
 
 if (renderImmediately) {
 renderPlot();
 }
 
 if (!renderImmediately || responsive) {
 // Set up observer for initial sizing or continuous monitoring
 var observer = new ResizeObserver(function(entries) {
 for (let entry of entries) {
 if (entry.contentBoxSize && 
 entry.contentBoxSize[0].inlineSize > 0) {
 if (!responsive && observer) {
 observer.disconnect();
 observer = null;
 }
 renderPlot();
 if (!responsive) {
 break;
 }
 }
 }
 });
 
 observer.observe(containerDiv);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 1.5 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 2.5 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 3.5 
 
 
 
 
 
 
 
 
 
 
 cos 
 
 
 
 
 
 
 
 
 sqrt 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ns/op